<a href="https://colab.research.google.com/github/sprksh-crypto/MachineLearningProjects/blob/Classification/customImputeLayerDefinition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#In 3.2 we've been asked to define a new class ImputerLayer creating a custom Keras preprocessing layer to replace NaN values with the mean
#across instances without NaN in this position. An adapt method must calculate the mean of the original non-NaN,
#values of each position in the vector input. Then the call method must replace the NaN values with the computed mean.

import tensorflow as tf

#Creating a custom Keras preprocessing layer.

class myImputer(tf.keras.layers.experimental.preprocessing.PreprocessingLayer):
  def __init__(self, **kwargs):
    super().__init__( **kwargs)
  def build(self,batch_input_shape):
    self.means=self.add_weight(name='means',shape=(batch_input_shape[-1]), initializer="zeros",trainable=False)
    super().build(batch_input_shape)

#Using call to replace the means.

  def call(self, X):
    return tf.where(tf.math.is_nan(X),self.means,X)

#Using adapt to calculate means of original non-NaNs.

  def adapt(self, dataset):
    self.build(dataset.element_spec.shape)
    sumOfNonNaNs=dataset.map(lambda z: tf.where(tf.math.is_nan(z), tf.zeros_like(z),z)).reduce(tf.zeros_like(self.means), lambda x,y: x+tf.reduce_sum(y,axis=0))
    numberOfNonNaNs=dataset.map(lambda z: tf.where(tf.math.is_nan(z),tf.zeros_like(z),tf.ones_like(z))).reduce(tf.zeros_like(self.means),lambda x,y: x+tf.reduce_sum(y,axis=0))
    self.means.assign(tf.math.divide(sumOfNonNaNs,numberOfNonNaNs))
  def compute_output_shape(self,batch_input_shape):
    return batch_input_shape

In [ ]:
#In 3.2.1 we've been asked to define a parse example method which converts the serialized example protobufs back into dictionaries with the
#keys tickers, weekday, hour, month, target then returns an ordered pair of the dictionary without the target and
#the target for use with supervised training.

import numpy as np

#First, all but the first column (tickers) are ints, let's deal with them.
#num_cols=mod_dataframe[1:]

feature_description= {"tickers": tf.io.FixedLenFeature([188], tf.float32, default_value=np.zeros(188)),
                      "weekday": tf.io.FixedLenFeature([], tf.int64, default_value=0),
                      "hour": tf.io.FixedLenFeature([], tf.int64, default_value=0),
                      "month": tf.io.FixedLenFeature([], tf.int64, default_value=0),
                      "target":tf.io.FixedLenFeature([], tf.int64, default_value=0)}

def parse_example(serialised):
  examples_parsed=tf.io.parse_example(serialised, feature_description)
  target=examples_parsed.pop('target')
  return examples_parsed, target

In [ ]:
#Need to mount drive again.

from google.colab import drive

#for authorisation
drive.mount('/content/drive')

#Drive files in "/content/drive/My Drive".
!ls "/content/drive/My Drive"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 appml-assignment1-dataset-v2.pkl   'ML HW 1.3'
'Assignment 1 ML.ipynb'		    'ML textbook code 1.ipynb'
 correct.tfrecord		     mySavedModel
 createSavedDataset		    'Physics Project'
 customImputeLayerDefinition.ipynb  'Physics Scripts'
 dataset.tfrecord		     Pics
'Essay Drafts'			     Untitled0.ipynb


In [ ]:
#3.2.2 Asks to load the dataset from the tfrecords file, batch it with a batch size of your selection, and process it with the parse
#example method. Cache the result in memory using the cache command from the data API.

filepath=['/content/drive/My Drive/dataset.tfrecord']
dataset=tf.data.TFRecordDataset(filepath)

In [ ]:
#3.2.3 asks to split the dataset into training, validation, and testing datasets.

datLen=dataset.reduce(0,lambda x,y: x+1)

n_valid=int(datLen.numpy()*.1)
n_test=int(datLen.numpy()*.2)
n_train=datLen.numpy()-n_valid-n_test

dataset_train=dataset.take(n_train).batch(4096).map(parse_example,num_parallel_calls=tf.data.AUTOTUNE).cache()
dataset_test=dataset.skip(n_train).take(n_test).batch(4096).map(parse_example,num_parallel_calls=tf.data.AUTOTUNE).cache()
dataset_validation=dataset.skip(n_train+n_test).take(n_valid).batch(4096).map(parse_example,num_parallel_calls=tf.data.AUTOTUNE).cache()



In [ ]:
#To check if I'm getting the right dataset.
#for el in manipulated_dataset.take(1):
#  print(el)

In [ ]:
#3.2.4 asks to create a series of 4 keras inputs for the 4 labels in the instance dictionary (tickers,weekday,hour,month), specifying
#the appropriate datatype and shape.

keras_inputs_Dict={'tickers': tf.keras.Input((188), dtype=tf.float32),
                   'weekday': tf.keras.Input((),dtype=tf.int64),
                   'hour': tf.keras.Input((),dtype=tf.int64),
                   'month': tf.keras.Input((),dtype=tf.int64)}

In [ ]:
#3.2.5 asks to import  the ImputerLayer, and adapt it to the training data for the tickers attributes.

myImp=myImputer()
myImp.adapt(dataset_train.map(lambda x,y: x['tickers']))

In [ ]:
#3.2.6 asks us to create a Keras preprocessing normalization and adapt it to output of the imputer applied to the training data
#for the tickers attributes.

#normalisation layer
normalizer=tf.keras.layers.Normalization(axis=-1)
#targets need to be removed, because we don't want adapt to learn the weights for it.
normalizer.adapt(dataset_train.map(lambda x,y:myImp(x['tickers'])))

In [ ]:
#3.2.7 "The tickers input should feed into the ImputerLayer followed by the Normalizer."